<a href="https://colab.research.google.com/github/lankipolo123/roadfixqc/blob/main/utility_pole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# YOLOv11n Utility Pole Detection Training
# Optimized for 90% Accuracy

# ============================================================================
# INSTALL AND SETUP
# ============================================================================
!pip install ultralytics
!pip install roboflow

import os
from ultralytics import YOLO
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ============================================================================
# DOWNLOAD UTILITY POLE DATASET
# ============================================================================
from roboflow import Roboflow
rf = Roboflow(api_key="SzttdelfmuWaCwAz2N5u")
project = rf.workspace("dequillaprojects").project("utility-pole-y8w7k")
version = project.version(1)
dataset = version.download("yolov11")

dataset_path = dataset.location
print(f"Dataset: {dataset_path}")

# Count images
train_count = len(os.listdir(f"{dataset_path}/train/images"))
val_count = len(os.listdir(f"{dataset_path}/valid/images"))
print(f"Training: {train_count} | Validation: {val_count}")

# ============================================================================
# TRAIN YOLOV11N - OPTIMIZED FOR UTILITY POLES
# ============================================================================
print("\n🚀 Training YOLOv11n for Utility Pole Detection...")

# Load YOLOv11n model
model = YOLO('yolo11n.pt')

# Train with optimized settings for utility poles
results = model.train(
    data=f'{dataset_path}/data.yaml',
    epochs=200,
    imgsz=832,          # Higher resolution for better detection
    batch=24,           # Nano can handle larger batches
    lr0=0.001,          # Learning rate
    lrf=0.01,           # Final learning rate factor
    optimizer='AdamW',  # Better optimizer
    cos_lr=True,        # Cosine learning rate scheduler
    patience=50,        # Early stopping
    save_period=10,     # Save every 10 epochs

    # Augmentation optimized for utility poles
    hsv_h=0.01,         # Slight hue changes
    hsv_s=0.4,          # Saturation changes (sky/lighting)
    hsv_v=0.3,          # Brightness changes (day/night)
    degrees=10,         # Rotation (poles are usually vertical)
    translate=0.1,      # Translation
    scale=0.3,          # Scale variation (distance from poles)
    shear=5,            # Less shear (poles are straight)
    fliplr=0.5,         # 50% horizontal flip
    flipud=0.0,         # No vertical flip
    mosaic=1.0,         # Mosaic augmentation
    mixup=0.1,          # Mixup augmentation

    # Loss weights
    box=7.5,            # Box loss weight
    cls=0.5,            # Classification loss
    dfl=1.5,            # DFL loss

    name='yolov11n_utility_pole',
    project='utility_pole_training',
    exist_ok=True,
    amp=True,           # Automatic Mixed Precision
    device='0' if torch.cuda.is_available() else 'cpu'
)

# ============================================================================
# VALIDATE AND CHECK RESULTS
# ============================================================================
print("\n📊 Validating model...")

val_results = model.val()
accuracy = val_results.box.map50 * 100

print(f"\n{'='*50}")
print(f"🎯 TRAINING RESULTS")
print(f"{'='*50}")
print(f"mAP50: {accuracy:.1f}%")
print(f"mAP50-95: {val_results.box.map*100:.1f}%")
print(f"Precision: {val_results.box.mp:.3f}")
print(f"Recall: {val_results.box.mr:.3f}")

# ============================================================================
# FINE-TUNING IF NEEDED (FOR 90% TARGET)
# ============================================================================
if accuracy < 90:
    print(f"\nCurrent: {accuracy:.1f}% | Target: 90%")
    print("🔧 Starting fine-tuning with higher resolution...")

    # Load best model for fine-tuning
    best_model_path = "utility_pole_training/yolov11n_utility_pole/weights/best.pt"
    fine_model = YOLO(best_model_path)

    # Fine-tune with higher resolution
    fine_results = fine_model.train(
        data=f'{dataset_path}/data.yaml',
        epochs=100,         # Fewer epochs for fine-tuning
        imgsz=1024,         # Higher resolution
        batch=12,           # Smaller batch for higher resolution
        lr0=0.0001,         # Much lower learning rate
        optimizer='AdamW',
        cos_lr=True,
        patience=30,
        mosaic=0.5,         # Reduce mosaic for fine-tuning
        mixup=0.05,         # Reduce mixup
        name='yolov11n_utility_pole_finetune',
        project='utility_pole_training',
        exist_ok=True
    )

    # Validate fine-tuned model
    final_results = fine_model.val()
    final_accuracy = final_results.box.map50 * 100

    print(f"🎉 Fine-tuned Accuracy: {final_accuracy:.1f}%")

    if final_accuracy > accuracy:
        accuracy = final_accuracy
        best_model_path = "utility_pole_training/yolov11n_utility_pole_finetune/weights/best.pt"
        model = fine_model

# ============================================================================
# FINAL RESULTS AND MODEL SAVING
# ============================================================================
print(f"\n{'='*50}")
print(f"🏆 FINAL RESULTS")
print(f"{'='*50}")
print(f"Final Accuracy: {accuracy:.1f}%")

if accuracy >= 90:
    print("🎉 SUCCESS! Achieved 90%+ accuracy!")
    status = "SUCCESS"
else:
    print(f"📈 Progress: {accuracy:.1f}% (Target: 90%)")
    status = "PROGRESS"

# Save final model with descriptive name
import shutil
final_model_name = f"utility_pole_yolov11n_{accuracy:.1f}percent_{status}.pt"
shutil.copy(best_model_path, final_model_name)
print(f"💾 Model saved as: {final_model_name}")

# ============================================================================
# TEST ON SAMPLE IMAGES
# ============================================================================
print("\n🔍 Testing on sample images...")

# Run predictions on validation images
val_image_dir = f"{dataset_path}/valid/images"
sample_images = os.listdir(val_image_dir)[:5]

for img_name in sample_images:
    img_path = os.path.join(val_image_dir, img_name)
    results = model.predict(
        source=img_path,
        conf=0.25,      # Confidence threshold
        save=True,      # Save results
        show_labels=True,
        show_conf=True
    )
    print(f"✅ Predicted: {img_name}")

print("\n🖼️ Prediction results saved in 'runs/detect/predict'")

# ============================================================================
# EXPORT MODEL (OPTIONAL)
# ============================================================================
print("\n📦 Exporting model...")

# Export to ONNX format for deployment
model.export(format='onnx')
print("Model exported to ONNX format")

print(f"\n{'='*50}")
print("🚀 UTILITY POLE DETECTION TRAINING COMPLETE!")
print(f"{'='*50}")
print(f"Dataset: {train_count} train + {val_count} val images")
print(f"Final Accuracy: {accuracy:.1f}%")
print(f"Model: {final_model_name}")

if accuracy < 90:
    print(f"\n💡 Tips to improve:")
    print("   • Check annotation quality")
    print("   • Add more diverse utility pole images")
    print("   • Try different lighting conditions")
    print("   • Increase training epochs")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 134.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with


Extracting Dataset Version Zip to utility-pole-1 in yolov11:: 100%|██████████| 48074/48074 [00:12<00:00, 3902.55it/s]


Dataset: /content/utility-pole-1
Training: 21024 | Validation: 1503

🚀 Training YOLOv11n for Utility Pole Detection...
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/utility-pole-1/data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.4, hsv_v=0.3, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov1

NameError: name 'best_model_path' is not defined

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful